In [11]:
import dotenv
#insert the location of your .env file here:
dotenv.load_dotenv('/home/eduardo/Documents/RW_github/cred/.env')
import geopandas as gpd
import glob
import os
import shutil
from shapely.geometry import Polygon, Point, mapping
import urllib.request
import sys
utils_path = os.path.join(os.path.abspath(os.getenv('PROCESSING_DIR')),'utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)
import util_files
import util_cloud
import util_carto
os.environ["GDAL_DATA"] = r'/home/eduardo/anaconda3/envs/po_env/lib/python3.6/site-packages/fiona/gdal_data/gcs.csv'
from zipfile import ZipFile
import logging
import pandas as pd

In [2]:
# name of table on Carto where you want to upload data
# this should be a table name that is not currently in use
dataset_name = 'cli_017_glacier_extent'

#logger.info('Executing script for dataset: ' + dataset_name)
# create a new sub-directory within your specified dir called 'data'
# within this directory, create files to store raw and processed data
data_dir = util_files.prep_dirs(dataset_name)
# download the data from the source

#url = 'http://www.glims.org/download/latest'
raw_data_file = os.path.join(data_dir, 'glacier_extent.zip')
#urllib.request.urlretrieve(url, raw_data_file)

# unzip source data
raw_data_file_unzipped = raw_data_file.split('.')[0]
zip_ref = ZipFile(raw_data_file, 'r')
zip_ref.extractall(raw_data_file_unzipped)
zip_ref.close()

In [26]:
'''
Process data
'''

# load in the polygon shapefile
shapefile_points = glob.glob(os.path.join(raw_data_file_unzipped,'glims_download_82381', 'glims_points.shp'))[0]
#shapefile_polygons = glob.glob(os.path.join(raw_data_file_unzipped,'glims_download_82381', 'glims_polygons.shp'))[0]

glims_points = gpd.read_file(shapefile_points)gdf['geometry'] = gdf['geometry'].to_crs(epsg=4326)
#glims_polygons = gpd.read_file(shapefile_polygons)

#Processing points shapefile
glims_points.rename(columns={"geometry": "the_geom"},inplace = True)
glims_points['the_geom'] = glims_points['the_geom'].apply(mapping)

#Processing polygons shapefile
glims_polygons.rename(columns={"length": "glacier_length", "geometry": "the_geom"},inplace = True)
#glims_polygons.drop(['glob_unc_x', 'glob_unc_y', 'loc_unc_x','loc_unc_y'], axis=1, inplace = True)
#glims_polygons['the_geom'] = glims_polygons['the_geom'].apply(mapping)


2021-02-12 16:59:14,978 - fiona._env - ERROR - Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [30]:
glims_polygons.head()

,line_type,anlys_id,glac_id,anlys_time,area,db_area,width,glacier_length,primeclass,min_elev,...,glac_stat,subm_id,release_dt,proc_desc,rc_id,geog_area,chief_affl,submitters,analysts,the_geom
0,debris_cov,121031.0,G006338E44942N,2013-03-25T00:00:00,0.0041,0.010639,0.0,61.1478,0.0,3040.0,...,exists,568.0,2013-12-25T14:08:34,Manual adjustment of the 1985 contour lines fo...,33.0,French Alps,Laboratoire de Glaciologie et Géophysique de l...,"Rabatel, Antoine","Rabatel, Antoine","POLYGON Z ((6.33947 44.94271 0.00000, 6.33948 ..."
1,debris_cov,121554.0,G006337E44931N,2013-03-20T00:00:00,1.9080,1.907390,0.0,3558.5700,0.0,2434.0,...,exists,569.0,2013-12-25T14:01:59,Automatic delineation using combination of SWI...,33.0,French Alps,Laboratoire de Glaciologie et Géophysique de l...,"Rabatel, Antoine","Rabatel, Antoine","POLYGON Z ((6.34182 44.93278 0.00000, 6.34179 ..."
2,debris_cov,121554.0,G006337E44931N,2013-03-20T00:00:00,1.9080,1.907390,0.0,3558.5700,0.0,2434.0,...,exists,569.0,2013-12-25T14:01:59,Automatic delineation using combination of SWI...,33.0,French Alps,Laboratoire de Glaciologie et Géophysique de l...,"Rabatel, Antoine","Rabatel, Antoine","POLYGON Z ((6.34330 44.93374 0.00000, 6.34329 ..."
3,debris_cov,121367.0,G006992E45913N,2013-03-25T00:00:00,7.6856,7.687140,0.0,5010.4600,0.0,2276.0,...,exists,568.0,2013-12-25T14:08:34,Manual adjustment of the 1985 contour lines fo...,33.0,French Alps,Laboratoire de Glaciologie et Géophysique de l...,"Rabatel, Antoine","Rabatel, Antoine","POLYGON Z ((6.98901 45.91748 0.00000, 6.98863 ..."
4,debris_cov,121128.0,G006340E45124N,2013-03-25T00:00:00,0.1238,0.123768,0.0,670.7820,0.0,2807.0,...,exists,568.0,2013-12-25T14:08:34,Manual adjustment of the 1985 contour lines fo...,33.0,French Alps,Laboratoire de Glaciologie et Géophysique de l...,"Rabatel, Antoine","Rabatel, Antoine","POLYGON Z ((6.34349 45.12189 0.00000, 6.34360 ..."


In [ ]:
print(glims_lines.columns)
print(glims_images.columns)
print(glims_points.columns)
print(glims_polygons.columns)
print(df.columns)

In [ ]:
df_cols = df.columns
polygons_cols = glims_polygons.columns
import numpy as np
main_list = np.setdiff1d(df_cols,polygons_cols)
main_list

In [ ]:
df = pd.read_csv('cli_017_glacier_extent.csv')